In [2]:
import cv2
from skimage.feature import local_binary_pattern
import matplotlib.pyplot as plt
from sklearn.svm import SVC
from sklearn.metrics import classification_report, accuracy_score
from sklearn.preprocessing import LabelEncoder
import os
from tqdm import tqdm 
import numpy as np
from sklearn.ensemble import RandomForestClassifier

In [2]:
def get_lbp(img_path, radius=1, n_points=8, method='ror'):
    image = cv2.imread(img_path, cv2.IMREAD_GRAYSCALE)
    image=cv2.resize(image, (224, 224), interpolation=cv2.INTER_AREA)
    lbp = local_binary_pattern(image, P=n_points, R=radius, method=method)
    return lbp

method uniform

In [6]:
import cv2
import numpy as np
from skimage.feature import local_binary_pattern

def get_lbp(img_path, grid_size=3, target_size=224):
    img = cv2.imread(img_path)
    img = cv2.resize(img, (target_size, target_size))
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    
    # Tính LBP
    lbp = local_binary_pattern(gray, P=8, R=1, method='uniform')
    
    h, w = lbp.shape
    bin_h = h // grid_size
    bin_w = w // grid_size
    
    features = []
    for i in range(grid_size):
        for j in range(grid_size):
            # Lấy ô con
            patch = lbp[i*bin_h:(i+1)*bin_h, j*bin_w:(j+1)*bin_w]
            # Tính histogram cho ô này
            hist, _ = np.histogram(patch.ravel(), bins=256, range=(0, 256), density=True)
            features.append(hist)
    
    return np.concatenate(features)  # shape: (grid_size*grid_size*10,)

In [7]:
len(get_lbp("/kaggle/input/hath-food/food dataset/Images/train/Banh canh/1.jpg"))

2304

In [9]:
BASE_PATH = "/kaggle/input/hath-food/food dataset/Images"

In [10]:
def load_dataset(root_path, subset="train"):

    data = []
    labels = []
    
    path = os.path.join(root_path, subset)
    if not os.path.exists(path):
        print(f"error in finding {path}")
        return np.array([]), np.array([])

    print(f"loading {subset}...")
    
    classes = os.listdir(path)
    
    for class_name in classes:
        class_dir = os.path.join(path, class_name)
        if not os.path.isdir(class_dir):
            continue
            
      
        files = os.listdir(class_dir)
        for file in tqdm(files, desc=class_name, leave=False):
            image_path = os.path.join(class_dir, file)
            
            hist = get_lbp(image_path)
            
            if hist is not None:
                data.append(hist)
                labels.append(class_name)
                
    return np.array(data), np.array(labels)

In [11]:
X_train, y_train_text = load_dataset(BASE_PATH, "train")
X_test, y_test_text = load_dataset(BASE_PATH, "test")

loading train...


Nem chua:  22%|██▏       | 82/379 [00:02<00:14, 20.98it/s]  Corrupt JPEG data: 9 extraneous bytes before marker 0xe2


loading test...


In [12]:
le = LabelEncoder()
y_train = le.fit_transform(y_train_text)
y_test = le.transform(y_test_text) 

In [13]:
model = SVC(kernel='rbf', C=10.0, gamma='scale', random_state=42, cache_size=100)
model.fit(X_train, y_train)

SVC(C=10.0, cache_size=100, random_state=42)

In [14]:
y_pred = model.predict(X_test)

acc = accuracy_score(y_test, y_pred)

In [ ]:
print(acc)

In [15]:
from sklearn.svm import SVC
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import classification_report
import numpy as np
print("Độ chính xác (và các metric) theo từng lớp:")
print(classification_report(y_test, y_pred, target_names=le.classes_))

Độ chính xác (và các metric) theo từng lớp:
                  precision    recall  f1-score   support

        Banh beo       0.21      0.14      0.17       129
    Banh bot loc       0.08      0.01      0.01       144
        Banh can       0.07      0.04      0.05       149
       Banh canh       0.18      0.14      0.16       193
      Banh chung       0.22      0.09      0.13       102
       Banh cuon       0.13      0.18      0.15       228
        Banh duc       0.13      0.02      0.03       133
        Banh gio       0.27      0.16      0.20       129
       Banh khot       0.17      0.11      0.14       167
         Banh mi       0.24      0.42      0.30       268
        Banh pia       0.20      0.10      0.13        89
        Banh tet       0.09      0.02      0.03       138
Banh trang nuong       0.21      0.14      0.17       159
        Banh xeo       0.15      0.31      0.20       235
      Bun bo Hue       0.12      0.42      0.19       306
 Bun dau mam tom       0.16

In [ ]:
import joblib
joblib.dump(model, 'svm_rbf_model.joblib')

In [16]:
classifier = RandomForestClassifier(n_estimators=500,min_samples_split=2,min_samples_leaf=2,max_depth=40)
classifier.fit(X_train, y_train)
y_pred = classifier.predict(X_test)

In [18]:
from sklearn.svm import SVC
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import classification_report
import numpy as np
print("Độ chính xác (và các metric) theo từng lớp:")
print(classification_report(y_test, y_pred, target_names=le.classes_))

Độ chính xác (và các metric) theo từng lớp:
                  precision    recall  f1-score   support

        Banh beo       0.34      0.12      0.17       129
    Banh bot loc       0.36      0.03      0.06       144
        Banh can       0.12      0.03      0.05       149
       Banh canh       0.13      0.08      0.10       193
      Banh chung       0.50      0.06      0.11       102
       Banh cuon       0.13      0.18      0.15       228
        Banh duc       0.06      0.01      0.01       133
        Banh gio       0.30      0.09      0.13       129
       Banh khot       0.20      0.05      0.08       167
         Banh mi       0.19      0.51      0.27       268
        Banh pia       0.23      0.15      0.18        89
        Banh tet       0.19      0.04      0.06       138
Banh trang nuong       0.24      0.18      0.21       159
        Banh xeo       0.14      0.30      0.19       235
      Bun bo Hue       0.11      0.47      0.18       306
 Bun dau mam tom       0.16